In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import os
os.getcwd()


'C:\\Users\\omprakash.bankol\\Desktop\\MARS_Russia\\Analytical Dataset Creation\\Code'

In [2]:
Nielsen_data = pd.read_csv(r"../Data/Raw Data/(GUMS )Neilsen_Complete_data_long_format.csv")

#drop SKUxWeek duplicates
Nielsen_data=Nielsen_data.drop_duplicates(subset=['SKU','Week'])
Nielsen_data.reset_index(inplace=True,drop=True)

sku_ppg =pd.read_csv(r"../Data/MappingFile/SKU_PPG_Mapping.csv")
week_mapping = pd.read_excel(r"../Data/MappingFile/Week Mapping.xlsx", sheet_name="Mapping")

In [3]:
Nielsen_data["Nielsen_Sales"] = Nielsen_data["1:Value (in 1000 RUR)"]*1000
Nielsen_data["Nielsen_Units"] = Nielsen_data["1:Units (in 1000 PACKS)"]*1000
cols = ["Manufacturer","Brand","SKU","Week","Nielsen_Sales","Nielsen_Units"]
Nielsen_data = Nielsen_data[cols]

In [4]:
#Date
week_mapping['Week_no'] = week_mapping['Week']
Nielsen_data['Year'] = pd.to_numeric(Nielsen_data['Week'].str.slice(2,6))
Nielsen_data['Week_no'] = pd.to_numeric(Nielsen_data['Week'].str.slice(7,9))
Nielsen_data['Week_no'] = "W"+pd.to_numeric(Nielsen_data['Week_no']).astype(str)
Nielsen_data = pd.merge(Nielsen_data,week_mapping, on=["Year", "Week_no"], how='left')

In [5]:
Nielsen_data['Date'] = pd.to_datetime(Nielsen_data['Week Start date'])

In [6]:
cols = ["Manufacturer","Brand","SKU","Year","Date","Nielsen_Sales","Nielsen_Units"]

#after 2017
Nielsen_data=Nielsen_data.loc[Nielsen_data['Year'] > 2017,]
Nielsen_data.reset_index(drop=True,inplace=True)
Nielsen_data.head(5)

,Manufacturer,Brand,SKU,Week_x,Nielsen_Sales,Nielsen_Units,Year,Week_no,Week Start date,Week_y,Month_Name,Month,Date
0,AO,AO,ANEKDOTY(K-ARTEL K.F/PODOLSK) CLASSIC B 1R 0.0...,W 2018 01,NaN,NaN,2018,W1,2017-12-31,W1,January,1,2017-12-31
1,AO,AO,ANEKDOTY(K-ARTEL K.F/PODOLSK) CLASSIC B 1R 0.0...,W 2018 02,NaN,NaN,2018,W2,2018-01-07,W2,January,1,2018-01-07
2,AO,AO,ANEKDOTY(K-ARTEL K.F/PODOLSK) CLASSIC B 1R 0.0...,W 2018 03,NaN,NaN,2018,W3,2018-01-14,W3,January,1,2018-01-14
3,AO,AO,ANEKDOTY(K-ARTEL K.F/PODOLSK) CLASSIC B 1R 0.0...,W 2018 04,NaN,NaN,2018,W4,2018-01-21,W4,January,1,2018-01-21
4,AO,AO,ANEKDOTY(K-ARTEL K.F/PODOLSK) CLASSIC B 1R 0.0...,W 2018 05,NaN,NaN,2018,W5,2018-01-28,W5,January,1,2018-01-28


In [7]:
######PPG Competitors##########
#comp_ppg = pd.read_excel(r"../Process/Comp/Nielsen_Competitors_Choco_Data_all_AS.xlsx")
comp_ppg =pd.read_excel(r"../Process/Competitor/Comp_decide/Nielsen_Competitors_GUM_Data_all_AS.xlsx")
comp_ppg.rename(columns={'Competitor_SKU':'SKU'},inplace=True)
print(comp_ppg.shape)
comp_ppg.dropna(subset=['Competitor_PPG'],inplace=True)
comp_ppg=comp_ppg.loc[(comp_ppg['Competitor_Weight_in_range']==1 )&(comp_ppg['Remarks']!='No data'), ]
comp_ppg.reset_index(inplace=True,drop=True)
comp_ppg.shape

(372, 16)


(67, 16)

In [8]:
#competitor sku count and sales share (value sales)
comp_ppg['Comp_SKU_count']=1
comp_ppg_sku = pd.DataFrame(comp_ppg.groupby(['PPG','Competitor_PPG'])['SKU_Sales_Share.2018','SKU_Sales_Share.2019','Comp_SKU_count'].sum())
comp_ppg_sku.head()
comp_ppg_sku.reset_index(inplace=True,drop=False)


In [9]:
#merge
comp_ppg_data =pd.merge(comp_ppg,Nielsen_data,on=['SKU'],how='left')
comp_ppg_data.head()

,PPG,PackSize,SKU,Competitor_Manufacturer,Competitor_Brand,Competitor_PC,Competitor_PackSize,Competitor_PPG,Competitor_Weight_in_range,Corr,...,Week_x,Nielsen_Sales,Nielsen_Units,Year,Week_no,Week Start date,Week_y,Month_Name,Month,Date
0,ORBIT XXL,20.4,CHUPA CHUPS(PERFETTI V.M.) BIG BABOL STRAWB. B...,PERFETTI,CHUPACHP,5,21.0,PERFETTI_CHUPACHP,1,0.655277,...,W 2018 01,NaN,NaN,2018,W1,2017-12-31,W1,January,1,2017-12-31
1,ORBIT XXL,20.4,CHUPA CHUPS(PERFETTI V.M.) BIG BABOL STRAWB. B...,PERFETTI,CHUPACHP,5,21.0,PERFETTI_CHUPACHP,1,0.655277,...,W 2018 02,NaN,NaN,2018,W2,2018-01-07,W2,January,1,2018-01-07
2,ORBIT XXL,20.4,CHUPA CHUPS(PERFETTI V.M.) BIG BABOL STRAWB. B...,PERFETTI,CHUPACHP,5,21.0,PERFETTI_CHUPACHP,1,0.655277,...,W 2018 03,NaN,NaN,2018,W3,2018-01-14,W3,January,1,2018-01-14
3,ORBIT XXL,20.4,CHUPA CHUPS(PERFETTI V.M.) BIG BABOL STRAWB. B...,PERFETTI,CHUPACHP,5,21.0,PERFETTI_CHUPACHP,1,0.655277,...,W 2018 04,NaN,NaN,2018,W4,2018-01-21,W4,January,1,2018-01-21
4,ORBIT XXL,20.4,CHUPA CHUPS(PERFETTI V.M.) BIG BABOL STRAWB. B...,PERFETTI,CHUPACHP,5,21.0,PERFETTI_CHUPACHP,1,0.655277,...,W 2018 05,NaN,NaN,2018,W5,2018-01-28,W5,January,1,2018-01-28


In [10]:
#merge
comp_ppg_data1=comp_ppg_data.groupby(['PPG','Competitor_PPG','Date'])['Nielsen_Sales','Nielsen_Units'].sum()
comp_ppg_data1.reset_index(inplace=True,drop=False)
comp_ppg_data1.head(2)


,PPG,Competitor_PPG,Date,Nielsen_Sales,Nielsen_Units
0,ORBIT OTC,MARS_5,2017-12-31,64400.0,1500.0
1,ORBIT OTC,MARS_5,2018-01-07,70100.0,1700.0


In [11]:
#ASP
comp_ppg_data1['Competitor_Price']=(comp_ppg_data1['Nielsen_Sales'])/(comp_ppg_data1['Nielsen_Units'])




In [12]:
#For summary
comp_ppg_data2=comp_ppg_data1.copy()
comp_ppg_data2.rename(columns={'Nielsen_Sales':'Competitor_Sales','Nielsen_Units':'Competitor_Units'},inplace=True)

Nielsen_data1 = pd.merge(Nielsen_data,sku_ppg, left_on=["SKU"], right_on=["Nielsen_SKU"],how='left')
Nielsen_data1 = Nielsen_data1[Nielsen_data1['PPG'].notna()]
Nielsen_data1.reset_index(drop=True,inplace=True)

#rolled up at PPG level
Nielsen_ppg_data=Nielsen_data1.groupby(['PPG','Date'])['Nielsen_Sales','Nielsen_Units'].sum()
Nielsen_ppg_data.reset_index(drop=False,inplace=True)
Nielsen_ppg_data['PPG_Price']=Nielsen_ppg_data['Nielsen_Sales']/Nielsen_ppg_data['Nielsen_Units']
Nielsen_ppg_data.rename(columns={'Nielsen_Sales':'PPG_Sales','Nielsen_Units':'PPG_Units'},inplace=True)
Nielsen_ppg_data.head()


,PPG,Date,PPG_Sales,PPG_Units,PPG_Price
0,ECLIPSE OTC,2017-12-31,7656000.0,315100.0,24.297049
1,ECLIPSE OTC,2018-01-07,11537100.0,500100.0,23.069586
2,ECLIPSE OTC,2018-01-14,10437800.0,438900.0,23.781727
3,ECLIPSE OTC,2018-01-21,8572300.0,346400.0,24.746824
4,ECLIPSE OTC,2018-01-28,7824900.0,304700.0,25.680670


In [13]:
df = pd.merge(comp_ppg_data2,Nielsen_ppg_data,on=['PPG','Date'],how='left')
df.head()

,PPG,Competitor_PPG,Date,Competitor_Sales,Competitor_Units,Competitor_Price,PPG_Sales,PPG_Units,PPG_Price
0,ORBIT OTC,MARS_5,2017-12-31,64400.0,1500.0,42.933333,38266900.0,1496600.0,25.569224
1,ORBIT OTC,MARS_5,2018-01-07,70100.0,1700.0,41.235294,44797400.0,1750200.0,25.595589
2,ORBIT OTC,MARS_5,2018-01-14,73500.0,1800.0,40.833333,48021600.0,1939700.0,24.757230
3,ORBIT OTC,MARS_5,2018-01-21,74900.0,1800.0,41.611111,53140800.0,2355800.0,22.557433
4,ORBIT OTC,MARS_5,2018-01-28,75500.0,1900.0,39.736842,67251100.0,3350600.0,20.071360


In [14]:
##Corrleation PPG X Competitor PPG (our quntity vs their price)
df1=df.groupby(['PPG','Competitor_PPG'])[['PPG_Units','Competitor_Price']].corr().unstack().iloc[:,1]
df1=pd.DataFrame(df1)
df1.reset_index(inplace=True,drop=False)
df1.columns = df1.columns.get_level_values(0)
df1.rename(columns={'PPG_Units':'Correlation'},inplace=True)
df1.head()

,PPG,Competitor_PPG,Correlation
0,ORBIT OTC,MARS_5,-0.086920
1,ORBIT OTC,MARS_DOUBLMNT,0.038747
2,ORBIT OTC,MARS_ECLIPSE OTC,-0.064764
3,ORBIT OTC,MARS_JUICYFRT,-0.268014
4,ORBIT OTC,MARS_WGSPRMNT,0.205130


In [15]:
#competitor sku count
df1=pd.merge(df1,comp_ppg_sku,on=['PPG','Competitor_PPG'],how='left')
df1.rename(columns={'SKU_Sales_Share.2018':'Comp_Sales_Share.2018','SKU_Sales_Share.2019':'Comp_Sales_Share.2019'},inplace=True)
df1.head(50)



,PPG,Competitor_PPG,Correlation,Comp_Sales_Share.2018,Comp_Sales_Share.2019,Comp_SKU_count
0,ORBIT OTC,MARS_5,-0.086920,1.185956e-03,5.708548e-04,4
1,ORBIT OTC,MARS_DOUBLMNT,0.038747,2.911677e-04,1.642858e-05,1
2,ORBIT OTC,MARS_ECLIPSE OTC,-0.064764,4.925450e-02,5.092726e-02,2
3,ORBIT OTC,MARS_JUICYFRT,-0.268014,1.179028e-02,8.019627e-03,4
4,ORBIT OTC,MARS_WGSPRMNT,0.205130,3.262446e-02,2.746870e-02,1
5,ORBIT OTC,MONDELEZ_DIROL OTC,0.015171,2.035837e-01,1.837991e-01,33
6,ORBIT OTC,MONDELEZ_DIROL OTC OTHERS,NaN,5.271594e-07,2.144190e-07,1
7,ORBIT OTC,MONDELEZ_STIMOROL,-0.084687,4.603858e-06,3.194740e-04,3
8,ORBIT OTC,PERFETTI_MENTOS,-0.032226,1.445588e-05,5.771067e-02,6
9,ORBIT XXL,MONDELEZ_DIROL XXL,-0.203617,4.657888e-02,4.591043e-02,5


In [16]:
##Export Summary file####
writer = pd.ExcelWriter(r"../Process/Competitor/Nielsen_PPG_Competitors_Summary_GUM.xlsx", engine='xlsxwriter')

# Write each dataframe to a different worksheet.
df1.to_excel(writer, sheet_name='Nielsen_PPG_Correlation')
df.to_excel(writer, sheet_name='Nielsen_PPG_Data')

# Close the Pandas Excel writer and output the Excel file.
writer.save()

In [17]:
#######For analytical Dataset########
#Long to wide
comp_ppg_data_wide=comp_ppg_data1.pivot_table(index=['PPG','Date'], columns='Competitor_PPG', values='Competitor_Price')
comp_ppg_data_wide.head()

Competitor_PPG           MARS_5  MARS_DOUBLMNT  MARS_ECLIPSE OTC  \
PPG       Date                                                     
ORBIT OTC 2017-12-31  42.933333      24.750000         24.297049   
          2018-01-07  41.235294      20.750000         23.069586   
          2018-01-14  40.833333      21.000000         23.781727   
          2018-01-21  41.611111      23.333333         24.746824   
          2018-01-28  39.736842      27.000000         25.680670   

Competitor_PPG        MARS_JUICYFRT  MARS_WGSPRMNT  MONDELEZ_DIROL OTC  \
PPG       Date                                                           
ORBIT OTC 2017-12-31      24.826435      22.632430           23.957918   
          2018-01-07      24.762246      22.500453           23.736803   
          2018-01-14      24.814480      22.549412           23.503626   
          2018-01-21      24.857803      22.521739           20.368567   
          2018-01-28      24.146624      23.379343           15.989346   

Competitor_PPG        MONDELEZ_DIROL OTC OTHERS  MONDELEZ_DIROL XXL  \
PPG       Date                                                        
ORBIT OTC 2017-12-31                        NaN                 NaN   
          2018-01-07                        NaN                 NaN   
          2018-01-14                        NaN                 NaN   
          2018-01-21                        inf                 NaN   
          2018-01-28                        NaN                 NaN   

Competitor_PPG        MONDELEZ_LOVEIS  MONDELEZ_STIMOROL  PERFETTI_CHUPACHP  \
PPG       Date                                                                
ORBIT OTC 2017-12-31              NaN                inf                NaN   
          2018-01-07              NaN                inf                NaN   
          2018-01-14              NaN          24.000000                NaN   
          2018-01-21              NaN          15.000000                NaN   
          2018-01-28              NaN                inf                NaN   

Competitor_PPG        PERFETTI_MENTOS  
PPG       Date                         
ORBIT OTC 2017-12-31        18.250000  
          2018-01-07        22.666667  
          2018-01-14        25.666667  
          2018-01-21        31.500000  
          2018-01-28        24.000000

In [18]:
#rename columns,ASP=Avg Selling price
comp_ppg_data_wide.rename(columns=lambda s: s+".Nielsen_ASP",inplace=True)
comp_ppg_data_wide.reset_index(inplace=True,drop=False)
comp_ppg_data_wide.columns          

Index(['PPG', 'Date', 'MARS_5.Nielsen_ASP', 'MARS_DOUBLMNT.Nielsen_ASP',
       'MARS_ECLIPSE OTC.Nielsen_ASP', 'MARS_JUICYFRT.Nielsen_ASP',
       'MARS_WGSPRMNT.Nielsen_ASP', 'MONDELEZ_DIROL OTC.Nielsen_ASP',
       'MONDELEZ_DIROL OTC OTHERS.Nielsen_ASP',
       'MONDELEZ_DIROL XXL.Nielsen_ASP', 'MONDELEZ_LOVEIS.Nielsen_ASP',
       'MONDELEZ_STIMOROL.Nielsen_ASP', 'PERFETTI_CHUPACHP.Nielsen_ASP',
       'PERFETTI_MENTOS.Nielsen_ASP'],
      dtype='object', name='Competitor_PPG')

In [19]:
comp_ppg_data_wide['PPG'].value_counts()

ORBIT OTC    117
ORBIT XXL    117
Name: PPG, dtype: int64

In [20]:
#remove time stamp
comp_ppg_data_wide['Date'] = pd.to_datetime(comp_ppg_data_wide['Date']).dt.date

In [21]:
comp_ppg_data_wide.to_excel(r"../Process/Competitor/Nielsen_PPG_Competitors_Data_GUM.xlsx",index=False)